In [ ]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher
from heterogt.utils.train import train_with_early_stopping
from heterogt.utils.seed import set_random_seed

In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"], 
    task_index = 2,
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]"],
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,   
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: stay


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_gender_sentences = ["[PAD]"] + [str(c) + "_" + gender for c in set(ehr_full_data["AGE"].values.tolist()) for gender in ["M", "F"]]
token_type_sentences = ["[PAD]"] + config.token_type
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
task_sentences = config.tasks
tokenizer = EHRTokenizer(token_type_sentences, age_gender_sentences, task_sentences, diag_sentences, 
                         med_sentences, lab_sentences, pro_sentences, special_tokens=config.special_tokens)
config.label_vocab_size = len(tokenizer.diag_voc.id2word)  # only for diagnosis
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_gender_vocab_size = tokenizer.token_number("age_gender")
print(f"Age and gender vocabulary size: {config.age_gender_vocab_size}")

Age and gender vocabulary size: 37


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 28.648477157360407 %
Percentage of READMISSION in test dataset: 40.1491116751269 %
Percentage of STAY>7 days in test dataset: 50.58692893401015 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task)

In [10]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [11]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "prauc"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "prauc"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [12]:
input_ids, token_types, adm_index, age_gender_ids, task_index, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age/Sex IDs shape:", age_gender_ids.shape)
print("Task Index:", task_index)
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 256])
Token Types shape: torch.Size([32, 256])
Admission Index shape: torch.Size([32, 256])
Age/Sex IDs shape: torch.Size([32, 7])
Task Index: 2
Labels shape: torch.Size([32, 1])


# Model Walkthrough

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData, Batch as HeteroBatch
from torch_geometric.nn import HeteroConv, GATConv
from heterogt.model.layer import TransformerEncoder

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [14]:
class DiseaseOccHetGNN(nn.Module):
    def __init__(self, d_model: int, heads: int = 1, dropout: float = 0.0):
        super().__init__()
        self.act = nn.GELU()

        # 第1层
        self.conv1 = HeteroConv({
            ('visit','contains','occ'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('occ','contained_by','visit'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('visit','next','visit'):       GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
        }, aggr='mean')

        # 第2层
        self.conv2 = HeteroConv({
            ('visit','contains','occ'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('occ','contained_by','visit'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('visit','next','visit'):       GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
        }, aggr='mean')
        self.lin = nn.Linear(d_model, d_model)

    def forward(self, hg):
        # x_dict: {'visit': [N_visit, d], 'occ': [N_occ, d]}
        x_dict = {'visit': hg['visit'].x, 'occ': hg['occ'].x}

        # 第1层：HeteroConv → Linear → GELU → Dropout
        x_dict = self.conv1(x_dict, hg.edge_index_dict)

        # 第2层：HeteroConv → Linear（末层通常不再加激活/随你需要）
        x_dict = self.conv2(x_dict, hg.edge_index_dict)
        x_dict = {k: self.lin(v) for k, v in x_dict.items()}

        return x_dict  # {'visit': [N_visit, d], 'occ': [N_occ, d]}

In [15]:
# multi-class classification task
class MultiPredictionHead(nn.Module):
    def __init__(self, hidden_size, label_size):
        super(MultiPredictionHead, self).__init__()
        self.cls = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), 
                nn.ReLU(), 
                nn.Linear(hidden_size, label_size)
            )

    def forward(self, input):
        return self.cls(input)
    
class BinaryPredictionHead(nn.Module):
    def __init__(self, hidden_size):
        super(BinaryPredictionHead, self).__init__()
        self.cls = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), 
                nn.ReLU(), 
                nn.Linear(hidden_size, 1)
            )
    def forward(self, input):
        return self.cls(input)

In [16]:
for i in range(len(train_dataset)):
    age_gender_ids = train_dataset[i][3]
    if len(age_gender_ids[0]) > 3:
        print(age_gender_ids)
        break
exp_i = i
id_seq = torch.concat([train_dataset[exp_i][0][0], torch.zeros(5, dtype=train_dataset[exp_i][0][0].dtype)], dim=0)
type_seq = torch.concat([train_dataset[exp_i][1][0], torch.zeros(5, dtype=train_dataset[exp_i][1][0].dtype)], dim=0)
visit_seq = torch.concat([train_dataset[exp_i][2][0], torch.zeros(5, dtype=train_dataset[exp_i][2][0].dtype)], dim=0)
age_sex = torch.concat([train_dataset[exp_i][3][0], torch.zeros(3, dtype=train_dataset[exp_i][3][0].dtype)], dim=0)

tensor([[11, 11, 11, 13]])


In [ ]:
class HeteroGT(nn.Module):
    def __init__(self, tokenizer, d_model, num_heads, num_layers, max_num_adms, device, task, use_hetero_graph):
        super(HeteroGT, self).__init__()
        self.device = device
        self.tokenizer = tokenizer
        self.max_num_adms = max_num_adms
        self.use_hetero_graph = use_hetero_graph
        self.global_vocab_size = len(self.tokenizer.vocab.word2id)
        self.age_sex_vocab_size = len(self.tokenizer.age_gender_voc.word2id)
        self.n_type = len(self.tokenizer.token_type_voc.word2id)
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.seq_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="all")[0] #0
        self.type_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="type")[0] #0
        self.adm_pad_id = 0
        self.age_sex_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="age_gender")[0] #0
        self.diag_type_id = 1
        self.visit_type_id = 5
        
        # embedding layers
        self.token_emb = nn.Embedding(self.global_vocab_size, d_model, padding_idx=self.seq_pad_id)
        self.type_emb = nn.Embedding(self.n_type + 1, d_model, padding_idx=self.type_pad_id) # n_type already have PAD, + 1 for visit
        self.adm_index_emb = nn.Embedding(self.max_num_adms + 1, d_model, padding_idx=self.adm_pad_id) # +1 for pad
        self.age_sex_emb = nn.Embedding(self.age_sex_vocab_size, d_model, padding_idx=self.age_sex_pad_id)
        self.task_emb = nn.Embedding(5, d_model, padding_idx=None)  # 5 task in total, task embedding, not used in this model
        
        # GNN
        self.het_gnn = DiseaseOccHetGNN(d_model)    

        # encoder transformer
        self.num_attn_heads = num_heads
        # enc_layer = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=self.num_attn_heads, batch_first=True, norm_first=True)
        # self.encoder = nn.TransformerEncoder(enc_layer, num_layers=self.num_layers, enable_nested_tensor=False)
        self.encoder = TransformerEncoder(d_model = self.d_model, num_heads = self.num_attn_heads, num_layers = self.num_layers)

        # prediction head
        if task in ["death", "stay", "readmission"]:
            self.cls_head = BinaryPredictionHead(self.d_model)
        else:
            self.cls_head = MultiPredictionHead(self.d_model, config.label_vocab_size)

    def run_graph(self, B, input_ids, token_types, adm_index, age_gender_index):
        """Run the heterogeneous graph neural network.

        Args:
            B (int): Batch size.
            input_ids (Tensor): Input token IDs. Shape of [B, L]
            token_types (Tensor): Token type IDs. Shape of [B, L]
            adm_index (Tensor): Admission index IDs. Shape of [B, L]
            age_gender_index (Tensor): Age and gender index IDs. Shape of [B, V_max]

        Returns:
            visit_emb_pad: Padded visit embeddings. Shape of [B, V_max, d]
            visit_pad_mask: Visit padding mask. Shape of [B, V_max]
            visit_index_pad: Padded visit index. Shape of [B, V_max]
            visit_type_ids_pad: Padded visit type IDs. Shape of [B, V_max], using self.visit_type_id and 0.
        """
        graphs = [] # contains heterogeneous graphs for each patient
        for p in range(B):
            hg_p = self.build_patient_graph(input_ids[p], token_types[p], adm_index[p], age_gender_index[p])
            graphs.append(hg_p)

        batch_graph = HeteroBatch.from_data_list(graphs).to(self.device)
        out = self.het_gnn(batch_graph)
        h_visit_all = out['visit']  # extract virtual visit node representations

        # 取出每个样本的 visit 表示序列（按我们在 build_patient_graph 中的保序构造）
        visit_emb_seq = []
        offset = 0
        for p in range(B):
            n_v = graphs[p]['visit'].num_nodes
            visit_emb_p = h_visit_all[offset:offset + n_v]  # [N_visit_p, d]
            offset += n_v
            visit_emb_seq.append(visit_emb_p)
            
        visit_emb_pad = []
        visit_pad_mask = []
        visit_index_pad = []
        for p in range(B):
            v = visit_emb_seq[p]                          # [N_visit_p, d]
            Np = v.size(0)
            if Np < self.max_num_adms:
                pad_len = self.max_num_adms - Np
                v_pad = torch.cat([v, torch.zeros(pad_len, self.d_model, device=self.device, dtype=v.dtype)], dim=0)
                m_pad = torch.cat([torch.zeros(Np, dtype=torch.bool, device=self.device), torch.ones(pad_len, dtype=torch.bool, device=self.device)], dim=0)
                i_pad = torch.cat([torch.arange(1, Np + 1, device=self.device), torch.full((pad_len,), self.adm_pad_id, dtype=torch.long, device=self.device)], dim=0)
            else:
                v_pad = v[:self.max_num_adms]
                m_pad = torch.zeros(self.max_num_adms, dtype=torch.bool, device=self.device)
                i_pad = torch.arange(1, self.max_num_adms + 1, device=self.device)
            visit_emb_pad.append(v_pad)      # [V_max, d]
            visit_pad_mask.append(m_pad)     # [V_max]
            visit_index_pad.append(i_pad)    # [V_max]

        visit_emb_pad  = torch.stack(visit_emb_pad,  dim=0)  # [B, V_max, d]
        visit_pad_mask = torch.stack(visit_pad_mask, dim=0)  # [B, V_max]
        visit_index_pad = torch.stack(visit_index_pad, dim=0)  # [B, V_max]

        visit_type_ids = torch.full((B, self.max_num_adms), self.visit_type_id, dtype=torch.long, device=self.device) # [B, V_max]
        visit_type_ids_pad = visit_type_ids * (~visit_pad_mask)

        return visit_emb_pad, visit_pad_mask, visit_index_pad, visit_type_ids_pad

    def build_patient_graph(self, id_seq: torch.Tensor, type_seq: torch.Tensor, adm_seq: torch.Tensor, age_sex: torch.Tensor):
        """Build a heterogeneous graph for a single patient.

        Args:
            id_seq (torch.Tensor): Sequence of token IDs of the current patient. Shape of [L]
            type_seq (torch.Tensor): Sequence of token types. Shape of [L]
            adm_seq (torch.Tensor): Sequence of admission IDs. Shape of [L]
            age_sex (torch.Tensor): Sequence of age and sex IDs. Shape of [V_max]

        Returns:
            a heterogeneous graph for the patient.
        """
        # build a graph just for one patient
        hg = HeteroData()
        occ_mask = (type_seq == self.diag_type_id) & (id_seq != self.seq_pad_id) # [L], a mask for the token types needed in the graph
        occ_pos = torch.nonzero(occ_mask, as_tuple=False).view(-1) # [L], seq position index for the token types needed in the graph
        N_occ = occ_pos.numel() # int, number of occurrences of the token types needed in the graph

        # build visit virtual nodes
        nonpad = adm_seq != self.adm_pad_id # [L], mask for non-pad tokens in the main seq
        adm_used = adm_seq[nonpad] # seq非pad部分
        adm_ids_unique, adm_lid_nonpad = torch.unique(adm_used, return_inverse=True) # adm_ids_unique: [V_max]
        adm_lid_full = torch.full_like(id_seq, fill_value=-1) # [L]
        adm_lid_full[nonpad] = adm_lid_nonpad
        N_visit = adm_ids_unique.numel() # should be V_max
        age_sex_nonpad = age_sex[age_sex!=self.age_sex_pad_id]
        assert N_visit == len(adm_ids_unique) == len(age_sex_nonpad)
        adm_x = self.age_sex_emb(age_sex_nonpad.to(self.device)) # [V_max, d]
        hg['visit'].x = adm_x
        hg['visit'].num_nodes = N_visit
        
        # build diag nodes
        gid_occ = id_seq[occ_pos]
        x_occ = self.token_emb(gid_occ) # [N_occ, d]
        hg['occ'].x = x_occ
        hg['occ'].num_nodes = N_occ

        # build edges between diag nodes and virtual visit nodes
        occ_adm_lid = adm_lid_full[occ_pos]
        e_v2o = torch.stack([occ_adm_lid, torch.arange(N_occ, device=self.device)], dim=0)
        e_o2v = torch.stack([torch.arange(N_occ, device=self.device), occ_adm_lid], dim=0)
        hg['visit','contains','occ'].edge_index = e_v2o
        hg['occ','contained_by','visit'].edge_index = e_o2v
        
        # build forward edges between virtual visit nodes
        if N_visit > 1:
            src = torch.arange(0, N_visit - 1, device=self.device)
            dst = torch.arange(1, N_visit, device=self.device)
            e_next = torch.stack([src, dst], dim=0) # [2, N_visit-1]
        else:
            e_next = torch.empty(2, 0, dtype=torch.long, device=self.device)
        hg['visit','next','visit'].edge_index = e_next
        return hg

    @staticmethod
    def build_attn_mask(token_types, forbid_map, num_heads):
        B, L = token_types.shape
        device = token_types.device
        
        if forbid_map == None:
            mask = torch.zeros((B, L, L), dtype=torch.bool, device=device)
        else:
            # 收集所有出现的 token 类型
            observed = torch.unique(token_types)
            for q_t, ks in forbid_map.items():
                observed = torch.unique(torch.cat([observed, torch.tensor([q_t] + list(ks), device=device)]))
            type_list = observed.sort().values
            t2i = {t.item(): i for i, t in enumerate(type_list)}  # Map token types to indices
            T = len(type_list)

            # 构造禁止矩阵 (T, T)，单向关系
            ban_table = torch.zeros((T, T), dtype=torch.bool, device=device)
            for q_t, ks in forbid_map.items():
                if q_t in t2i:
                    qi = t2i[q_t]
                    for k_t in ks:
                        if k_t in t2i:
                            ban_table[qi, t2i[k_t]] = True  # 只设置 q -> k 的禁止

            # 向量化映射 token_types 到类型索引
            mapping = torch.zeros_like(type_list, dtype=torch.long, device=device)
            for t, i in t2i.items():
                mapping[type_list == t] = i
            q_idx = mapping[torch.searchsorted(type_list, token_types.unsqueeze(-1))]
            k_idx = mapping[torch.searchsorted(type_list, token_types.unsqueeze(-2))]

            # 查询 ban_table 得到 (B, L, L)
            mask = ban_table[q_idx, k_idx].to(torch.bool)
        
        # 扩展到 num_heads
        mask = mask.unsqueeze(1).expand(B, num_heads, L, L)
        mask = mask.reshape(B * num_heads, L, L)
        return mask
    
    def forward(self, input_ids, token_types, adm_index, age_gender_index, task_id):
        # input_ids, [B, L], pad_id = 0, token_types, [B, L], pad_id = 0, adm_index, [B, L], pad_id = 0, 
        # age_gender_index, [B, V_max], task_id, [1]
        B, L = input_ids.shape
        task_id = torch.full((B,), task_id, dtype=torch.long, device=self.device) # [1] -> [B]
        # 基础表示
        token_embed = self.token_emb(input_ids)  # [B, L, d]
        seq_pad_mask = (input_ids == self.seq_pad_id) # [B, L], bool

        forbid_map = None # a dict, encode the forbidden attentions,
        # e.g., {3: [1, 2, 4]} means that token type 3 cannot attend to token types 1, 2, and 4, it is uni-directional
        if self.use_hetero_graph: # use graph as middle layer
            # get visit embed and mask
            visit_emb_pad, visit_pad_mask, visit_index_pad, visit_type_ids_pad = self.run_graph(B, input_ids, token_types, adm_index, age_gender_index)
            # elongate the main sequence with visit information
            token_embed = torch.concat([token_embed, visit_emb_pad], dim=1)  # [B, L+V, d]
            adm_index = torch.concat([adm_index, visit_index_pad], dim=1)  # [B, L+V]
            token_types = torch.concat([token_types, visit_type_ids_pad], dim=1) # [B, L+V]
            seq_pad_mask = torch.concat([seq_pad_mask, visit_pad_mask], dim=1) # [B, L+V]
            
        adm_emb = self.adm_index_emb(adm_index) # [B, L+V, d]
        token_type_emb = self.type_emb(token_types) # [B, L+V, d]
        x = token_embed + adm_emb + token_type_emb # [B, L+V, d]
        task_id_emb = self.task_emb(task_id).unsqueeze(1) # [B, 1, d]
        x = torch.concat([task_id_emb, x], dim=1) # [B, 1+L+V, d]

        # mask
        task_pad_mask = torch.zeros((B, 1), dtype=torch.bool, device=self.device)
        src_key_padding_mask = torch.concat([task_pad_mask, seq_pad_mask], dim=1)  # [B, 1+L+V]
        attn_mask = self.build_attn_mask(torch.concat([torch.full((B, 1), -1, device=self.device), token_types], dim=1), 
                                         forbid_map=forbid_map, 
                                         num_heads=self.num_attn_heads)
        assert attn_mask.dtype == src_key_padding_mask.dtype, f"attn_mask dtype ({attn_mask.dtype}) and src_key_padding_mask dtype ({src_key_padding_mask.dtype}) must match"
        
        # ===== Transformer 编码（batch_first=True） =====
        h = self.encoder(src=x, src_key_padding_mask=src_key_padding_mask, mask=attn_mask)   # [B, 1+L(+V), d]

        # ===== 分类：取 CLS（task 位） =====
        logits = self.cls_head(h[:, 0, :])  # [B, label_size]
        return logits

In [18]:
final_metrics = {"precision":[],"recall":[],"f1":[],"auc":[],"prauc":[]}
for i in range(5):
    model = HeteroGT(tokenizer, d_model=128, num_heads=4, num_layers=2, max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, use_hetero_graph=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    for key in final_metrics.keys():
        final_metrics[key].append(best_test_metric[key])

Running inference: 100%|██████████| 197/197 [00:04<00:00, 44.77it/s]


Validation: {'precision': 0.7570834907413786, 'recall': 0.6284101599227707, 'f1': 0.6867717565212385, 'auc': 0.7894704531929437, 'prauc': 0.7930792404377097}
Test:      {'precision': 0.7393655371278322, 'recall': 0.6431483223560892, 'f1': 0.6879087657748875, 'auc': 0.7790617023707599, 'prauc': 0.7860167804328437}


Running inference: 100%|██████████| 197/197 [00:04<00:00, 44.62it/s]


Validation: {'precision': 0.6873655913960017, 'recall': 0.8018187519573478, 'f1': 0.7401939449477855, 'auc': 0.8036371723570359, 'prauc': 0.8156585775057466}
Test:      {'precision': 0.6744680851045892, 'recall': 0.7952336155509714, 'f1': 0.7298891877212547, 'auc': 0.7891822159540194, 'prauc': 0.8022493903051733}


Running inference: 100%|██████████| 197/197 [00:04<00:00, 47.47it/s]


...

Validation: {'precision': 0.7386046511605004, 'recall': 0.7469426152375449, 'f1': 0.7427502288612653, 'auc': 0.8201197564240019, 'prauc': 0.8266756263643691}
Test:      {'precision': 0.7437170338171154, 'recall': 0.7516462840992423, 'f1': 0.7476606313047331, 'auc': 0.8244155899065154, 'prauc': 0.83265665821936}


Running inference: 100%|██████████| 197/197 [00:04<00:00, 49.10it/s]


Validation: {'precision': 0.7497618291497309, 'recall': 0.7403574788311685, 'f1': 0.7450299729088612, 'auc': 0.8209445939658203, 'prauc': 0.8275478683388723}
Test:      {'precision': 0.7448494453225203, 'recall': 0.7369081216659238, 'f1': 0.7408574981503931, 'auc': 0.8240953176222238, 'prauc': 0.8315355793010908}


Running inference: 100%|██████████| 197/197 [00:04<00:00, 48.75it/s]

Validation: {'precision': 0.7046703296683938, 'recall': 0.804327375350253, 'f1': 0.7512080781943039, 'auc': 0.813847544863697, 'prauc': 0.8171968265078629}
Test:      {'precision': 0.7081627081607749, 'recall': 0.8134211351495346, 'f1': 0.7571511917525988, 'auc': 0.8201537991500678, 'prauc': 0.8255005493252348}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.7436868686845212, 'recall': 0.7387895892106028, 'f1': 0.7412301350008688, 'auc': 0.8225400635736289, 'prauc': 0.8331693279929278}
Corresponding test performance:
{'precision': 0.7416744475544922, 'recall': 0.747256193161658, 'f1': 0.7444548528546644, 'auc': 0.8234368039815838, 'prauc': 0.8332993994456532}


In [19]:
# print the mean and std of the final metrics
print("\nFinal Metrics:")
for key in final_metrics.keys():
    mean_value = np.mean(final_metrics[key])
    std_value = np.std(final_metrics[key])
    print(f"{key}: {mean_value:.4f} ± {std_value:.4f}")


Final Metrics:
precision: 0.7416 ± 0.0494
recall: 0.7398 ± 0.0774
f1: 0.7353 ± 0.0177
auc: 0.8202 ± 0.0106
prauc: 0.8307 ± 0.0125


In [20]:
final_metrics = {"precision":[],"recall":[],"f1":[],"auc":[],"prauc":[]}
for i in range(5):
    model = HeteroGT(tokenizer, d_model=128, num_heads=4, num_layers=2, max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, use_hetero_graph=False).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    for key in final_metrics.keys():
        final_metrics[key].append(best_test_metric[key])

Epoch 001:   0%|          | 0/98 [00:00<?, ?it/s, loss=0.7584]

Running inference: 100%|██████████| 197/197 [00:00<00:00, 223.47it/s]


Validation: {'precision': 0.8118489583280479, 'recall': 0.3910316713691094, 'f1': 0.5278306834403983, 'auc': 0.7558063911019024, 'prauc': 0.7659564036686408}
Test:      {'precision': 0.8089053803289493, 'recall': 0.41015992474001206, 'f1': 0.5443195960320538, 'auc': 0.7618889068411832, 'prauc': 0.7712677424020319}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 223.23it/s]


Validation: {'precision': 0.6325842696614998, 'recall': 0.8827218563785427, 'f1': 0.7370074568439654, 'auc': 0.8057807251356368, 'prauc': 0.8201910613932202}
Test:      {'precision': 0.628500451669764, 'recall': 0.8726873628069217, 'f1': 0.7307338798618984, 'auc': 0.8041602680160082, 'prauc': 0.8224039108805503}


...

Validation: {'precision': 0.7150355047832201, 'recall': 0.7262464722460764, 'f1': 0.7205973814324103, 'auc': 0.7939034463329117, 'prauc': 0.8080383259679054}
Test:      {'precision': 0.7092952612372135, 'recall': 0.7322044528042264, 'f1': 0.7205678086080501, 'auc': 0.7942108884523293, 'prauc': 0.8082859150046365}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 223.96it/s]


Validation: {'precision': 0.7603423680429374, 'recall': 0.6685481342092551, 'f1': 0.7114967412221651, 'auc': 0.8040099291972999, 'prauc': 0.8146837112254438}
Test:      {'precision': 0.7536433032590089, 'recall': 0.6810912511737814, 'f1': 0.7155328561535681, 'auc': 0.8058019969326744, 'prauc': 0.8124746080000023}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 223.40it/s]

Validation: {'precision': 0.7292782855318964, 'recall': 0.72561931639785, 'f1': 0.7274441949348717, 'auc': 0.7958944300382131, 'prauc': 0.8057866634544943}
Test:      {'precision': 0.7176724137908939, 'recall': 0.7309501411077738, 'f1': 0.7242504222158178, 'auc': 0.7951361194958391, 'prauc': 0.8034014652546613}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.7327044025134192, 'recall': 0.7306365631836607, 'f1': 0.7316690168221748, 'auc': 0.8060646131281342, 'prauc': 0.8198914571198107}
Corresponding test performance:
{'precision': 0.7264325323452667, 'recall': 0.739416745058829, 'f1': 0.7328671278652475, 'auc': 0.8088750102554578, 'prauc': 0.8196527921959437}


In [21]:
# print the mean and std of the final metrics
print("\nFinal Metrics:")
for key in final_metrics.keys():
    mean_value = np.mean(final_metrics[key])
    std_value = np.std(final_metrics[key])
    print(f"{key}: {mean_value:.4f} ± {std_value:.4f}")


Final Metrics:
precision: 0.7001 ± 0.0380
recall: 0.7974 ± 0.0504
f1: 0.7432 ± 0.0109
auc: 0.8139 ± 0.0069
prauc: 0.8266 ± 0.0058
